## Importation des bibliothèques

In [1]:
# Importer les librairies
import numpy as np
import shap
import pandas  as pd
from sklearn.feature_selection import mutual_info_classif, RFE
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Importation du jeu de données

In [2]:
# ====== Importer le jeu de données ======
df= pd.read_excel(r"D:\Projects\IT\Data Science & IA\Developpement_Tableau_de_Bord_de_la_Chaine_Approvisionnement_Power_BI\data\preprocessed_data_supply.xlsx")
print("Jeu de données importé ✅✅")

Jeu de données importé ✅✅


## Supression des colonnes inutiles et exportation

In [3]:
# Gestion des colonnes dates
df['Estimated Delivery Date'] = pd.to_datetime(df['Estimated Delivery Date'], format='%d/%m/%Y', errors='coerce')
df['Estimated_day'] = df['Estimated Delivery Date'].dt.day

# Suppression des colonnes inutiles
colonnes_a_effacer = ["ID","Delivery Date","Item",'Estimated Delivery Date',"Delivery Delay","Supplier","Warehouse"]
df = df.drop(columns=colonnes_a_effacer)
df['Estimated_day']

# Exportation de la base de données
df.to_excel(r"D:\Projects\IT\Data Science & IA\Developpement_Tableau_de_Bord_de_la_Chaine_Approvisionnement_Power_BI\data\supply_data.xlsx",index=False)
print("Jeu de données exporté ✅✅")

Jeu de données exporté ✅✅


## Encodages des features catégorielles

In [4]:
# Gestions des colonnes catégorielles
cat_col = df.select_dtypes(include=['object']).columns.difference(['Delivery Status', 'Delivery Delay','Item']).tolist()

encoded_dfs = []
for col in cat_col:
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    df[col] = df[col].astype(str)
    encoded = pd.DataFrame(
        encoder.fit_transform(df[[col]]),
        columns=encoder.get_feature_names_out([col]),
        index=df.index
    )
    encoded_dfs.append(encoded)

# Fusionner les colonnes encodées et supprimer les originales
df = df.drop(columns=cat_col)
if encoded_dfs:
    df = pd.concat([df] + encoded_dfs, axis=1)
    


## Application de Mutual Information

In [5]:
# Préparation des données
x = df.drop(columns=['Delivery Status'])
y = df['Delivery Status']

# ====== Initialisation et application de Mutual Information ======
mutual = mutual_info_classif(x, y, discrete_features='auto')
mutual_df = pd.DataFrame({
    "Features": x.columns,
    "MI Scores": mutual
}).sort_values(by="MI Scores", ascending=False)

mutual_selected = mutual_df[mutual_df['MI Scores'] > 0.01]

# Affichage
print("Les features pertinentes")
print(mutual_selected)

Les features pertinentes
               Features  MI Scores
2   Transportation Cost   0.025475
8           Region_West   0.021321
10   Restock Needed_Yes   0.015480
1                 Sales   0.010628


## Application du RFE - Choix des features pertinentes

In [6]:
# Division des données en entraînement et en test
x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=42,test_size=0.2)

# Initialisation et entraînement du modèle
rf = RandomForestClassifier(random_state=42)
model = rf.fit(x_train, y_train)

# Entrapineent du modèle RFE
rfe = RFE(rf, n_features_to_select=10, step=1)
selector = rfe.fit(x_train,y_train)

# Affichage des features pertinentes
features_slected = x_train.columns[selector.support_]
print("Features sélectionnées après RFE", features_slected)

Features sélectionnées après RFE Index(['Stock Level', 'Sales', 'Transportation Cost',
       'Delivery Urgency_On Time', 'Region_East', 'Region_North',
       'Region_South', 'Region_West', 'Restock Needed_No',
       'Restock Needed_Yes'],
      dtype='object')


## Application de SHAP

In [7]:
# Analayse SHAP avec TreeExplainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_test)

# Gestion des SHAP values
if isinstance(shap_values, list):
    shap_values_aggregated = np.mean([np.abs(sv) for sv in shap_values], axis=0)
else: 
    shap_values_aggregated = np.abs(shap_values)

# Conversion explicite des noms de colonnes en array
columns_array = np.array(x_test.columns)
shap_importance = shap_values_aggregated.mean(axis=0)
shap_features = columns_array[np.argsort(-shap_importance)[:10]] #Top 10 features SHAP

# Affichage
print("\nTop features selon SHAP :")
print(shap_features)


Top features selon SHAP :
[['Sales' 'Transportation Cost' 'Stock Level']
 ['Sales' 'Stock Level' 'Transportation Cost']
 ['Sales' 'Stock Level' 'Transportation Cost']
 ['Stock Level' 'Sales' 'Transportation Cost']
 ['Stock Level' 'Sales' 'Transportation Cost']
 ['Transportation Cost' 'Sales' 'Stock Level']
 ['Transportation Cost' 'Sales' 'Stock Level']
 ['Transportation Cost' 'Sales' 'Stock Level']
 ['Sales' 'Transportation Cost' 'Stock Level']
 ['Transportation Cost' 'Sales' 'Stock Level']]


## Mise au propre du dataset

In [8]:
# Importer le jeu de données
dataset = pd.read_excel(r"D:\Projects\IT\Data Science & IA\Developpement_Tableau_de_Bord_de_la_Chaine_Approvisionnement_Power_BI\data\supply_data.xlsx")
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Delivery Status      700 non-null    int64  
 1   Stock Level          700 non-null    int64  
 2   Sales                700 non-null    int64  
 3   Transportation Cost  700 non-null    float64
 4   Restock Needed       700 non-null    object 
 5   Region               700 non-null    object 
 6   Delivery Urgency     700 non-null    object 
 7   Estimated_day        700 non-null    int64  
dtypes: float64(1), int64(4), object(3)
memory usage: 43.9+ KB


## Conversion des colonnes catégorielles encodées en colonnes brutes

In [9]:
dataset['Delivery Status'] = dataset['Delivery Status'].replace({
    0 : "On time",
    1 : "Late",
    -1 : "Missing"
})

print("Ciblé encodée, désormais catégorisée ✅✅")

Ciblé encodée, désormais catégorisée ✅✅


In [10]:
import numpy as np

mask = dataset["Delivery Urgency"] == 'On Time'  # détecte les lignes à modifier
indices = dataset[mask].sample(frac=0.3, random_state=42).index  # 30% des "On Time"

# Générer une répartition aléatoire entre "Late" et "On Schedule"
remplacement = np.random.choice(['Late', 'On Schedule'], size=len(indices), replace=True)

# Appliquer les nouvelles valeurs
dataset.loc[indices, "Delivery Urgency"] = remplacement

print("Création aléatoire réussie ✅✅")
dataset["Delivery Urgency"].value_counts()

Création aléatoire réussie ✅✅


Delivery Urgency
On Time        490
On Schedule    107
Late           103
Name: count, dtype: int64

## Création des données aléatoires dans la colonne

In [11]:
# Détecter les lignes à modifier (exemple : 30% des valeurs)
indices = dataset.sample(frac=0.3, random_state=42).index  

# Générer de nouvelles valeurs aléatoires entre 1 et 17
remplacement = np.random.randint(1, 18, size=len(indices))  # randint prend [low, high), donc 18 pour inclure 17

# Appliquer les nouvelles valeurs
dataset.loc[indices, "Estimated_day"] = remplacement

# Afficher la répartition mise à jour
print("Creation réussie ✅✅")

Creation réussie ✅✅


In [12]:
# Exportation de la base de données
dataset.to_excel(r"D:\Projects\IT\Data Science & IA\Developpement_Tableau_de_Bord_de_la_Chaine_Approvisionnement_Power_BI\data\supply-chain.xlsx",index=False)
print("Jeu de données brut importé ✅✅")

Jeu de données brut importé ✅✅
